# Estadística Aplicada
# Actividad 6 parte A

Cecilia Sarahí Gutiérrez Padilla 1953437 Gpo 43

Cada vez que un empleado renuncia a una empresa “X”, a esta empresa le toma como mínimo 6 meses contratar un remplazo, más 3 meses de capacitación y un mes de adaptación. Tal situación implica pérdida de tiempo y recursos por los siguientes motivos:

a) Cuando un empleado se va, sus responsabilidades deben ser redistribuidas entre los empleados restantes. Esto puede generar una carga de trabajo adicional para aquellos que quedan, lo que puede llevar a la fatiga.

b) Reemplazar a un empleado puede ser costoso. Incluye gastos en reclutamiento, selección, capacitación y adaptación.

c) La rotación frecuente de empleados puede llevar a interrupciones en el flujo de trabajo, ya que los nuevos empleados necesitan tiempo para adaptarse y alcanzar el mismo nivel de productividad que sus predecesores.

Por tal razón la empresa “X” desea conocer las razones que hacen que sus empleados renuncien, para tal fin le ha proporcionado una base de datos llamada deserción_laboral que mide el tiempo en semestres desde que un empleado entró a trabajara la empresa “X” hasta que renuncia. Donde el evento es renunciar.

**renunciar**: 1: renunció, 0: censurado\
**semestre**: Semestres que ha trabajado en la empresa\
**satisfacción**: Que tan satisfecho se encuentra la persona con su trabajo en una escala de 0 a 10\
**evaluación**: Que tan bien realiza su trabajo la persona en una escala de 0 a 10\
**proyectos**: Número de proyectos asignados al empleado\
**promedio_horas_mes**: Promedio de horas mensuales trabajadas\
**accidentes**: 1: ha tenido un accidente dentro de la empresa, 0: no ha tenido accidentes dentro de la empresa\
**ascenso_5years**: 1: si el empleado ha sido ascendido durante los últimos 5 años, 0: si no ha sido ascendido durante los últimos 5 años\
**departamento**: Nombre del departamento en el que trabaja\
**salario**: Clasificación del salario en alto, medio y bajo

#### 1. Importa las librerías básicas y especializadas que consideres necesarias

In [1]:
## Librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Liberías especializadas
from lifelines import CoxPHFitter, ExponentialFitter, WeibullFitter, LogNormalFitter, WeibullAFTFitter, LogNormalAFTFitter
import seaborn as sns

#### 2. Importa la base de datos https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv

In [3]:
## Datos
desercion_laboral = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')
desercion_laboral

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
14994,4.0,5.7,2,151,3,0,1,0,soporte,bajo
14995,3.7,4.8,2,160,3,0,1,0,soporte,bajo
14996,3.7,5.3,2,143,3,0,1,0,soporte,bajo
14997,1.1,9.6,6,280,4,0,1,0,soporte,bajo


#### 3. Depurar la base de datos y estadísticas descriptivas

##### a) Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar

In [4]:
desercion_laboral.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [5]:
## Eliminar missing values departamento, puesto que eliminar las filas con valores perdidos no sería una pérdida significativa de datos
desercion_laboral = desercion_laboral.dropna()
desercion_laboral.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64

##### b) Verificar que no haya valores duplicados con dataframe.duplicated().sum() en caso de haber valores duplicados eliminarlos con dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)

In [6]:
## Verificar valores duplicados
desercion_laboral.duplicated().sum()

2982

In [7]:
## Eliminar valores duplicados
desercion_laboral = desercion_laboral.drop_duplicates(keep='first').reset_index(drop=True)
desercion_laboral.duplicated().sum()

0

##### c) Verificar que todas las variables sean numéricas, si no son numéricas hacerlas numéricas, en el caso de las categóricas convertirlas en dummies y conservar n-1 dummies

In [8]:
## Verificar que las variables sean numéricas
desercion_laboral.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [9]:
## convertir departamento en dummies
dummies_dep = pd.get_dummies(desercion_laboral['departamento'], prefix='dept').astype(int)
dummies_dep


,dept_Investigación,dept_administración,dept_contabilidad,dept_desarrollo de producto,dept_informática,dept_mantenimiento,dept_mercadotecnia,dept_recursos humanos,dept_soporte,dept_ventas
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
11986,0,0,0,1,0,0,0,0,0,0
11987,0,0,0,1,0,0,0,0,0,0
11988,0,0,0,1,0,0,0,0,0,0
11989,0,0,0,1,0,0,0,0,0,0


In [10]:
dummies_dep = dummies_dep[['dept_Investigación', 'dept_administración', 'dept_contabilidad', 'dept_desarrollo de producto', 'dept_informática', 'dept_mantenimiento', 'dept_mercadotecnia', 'dept_recursos humanos', 'dept_soporte']]
desercion_laboral = pd.concat([desercion_laboral, dummies_dep], axis=1)
desercion_laboral = desercion_laboral.drop('departamento', axis=1)

In [11]:
## convertir salario en dummies
dummies_sal = pd.get_dummies(desercion_laboral['salario'], prefix='sal').astype(int)
dummies_sal

,sal_alto,sal_bajo,sal_medio
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0
...,...,...,...
11986,0,1,0
11987,0,1,0
11988,0,1,0
11989,0,1,0


In [12]:
dummies_sal = dummies_sal[['sal_alto', 'sal_bajo']]
desercion_laboral = pd.concat([desercion_laboral, dummies_sal], axis=1)
desercion_laboral = desercion_laboral.drop('salario', axis=1)

In [13]:
desercion_laboral.isna().sum()

satisfaccion                   0
evaluacion                     0
proyectos                      0
promedio_horas_mes             0
semestre                       0
accidente                      0
renunciar                      0
ascenso_5years                 0
dept_Investigación             0
dept_administración            0
dept_contabilidad              0
dept_desarrollo de producto    0
dept_informática               0
dept_mantenimiento             0
dept_mercadotecnia             0
dept_recursos humanos          0
dept_soporte                   0
sal_alto                       0
sal_bajo                       0
dtype: int64

In [14]:
desercion_laboral

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dept_Investigación,dept_administración,dept_contabilidad,dept_desarrollo de producto,dept_informática,dept_mantenimiento,dept_mercadotecnia,dept_recursos humanos,dept_soporte,sal_alto,sal_bajo
0,3.8,5.3,2,157,3,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,8.0,8.6,5,262,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1.1,8.8,7,272,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,7.2,8.7,5,223,5,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,3.7,5.2,2,159,3,0,1,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11986,3.7,4.7,2,152,3,0,1,0,0,0,0,1,0,0,0,0,0,0,1
11987,8.4,9.9,4,267,5,0,1,0,0,0,0,1,0,0,0,0,0,0,1
11988,4.1,4.6,2,151,3,0,1,0,0,0,0,1,0,0,0,0,0,0,1
11989,7.6,9.2,4,239,5,0,1,0,0,0,0,1,0,0,0,0,0,0,1


##### d) Calcular estadísticas descriptivas

In [15]:
desercion_laboral.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dept_Investigación,dept_administración,dept_contabilidad,dept_desarrollo de producto,dept_informática,dept_mantenimiento,dept_mercadotecnia,dept_recursos humanos,dept_soporte,sal_alto,sal_bajo
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.057877,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.082562,0.478692
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.233520,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.275230,0.499567
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


- **Satisfacción**: Los empleados muestran un nivel de satisfacción moderado en promedio, con una media de alrededor de 6.30. La variabilidad en los niveles de satisfacción se refleja en una desviación estándar de aproximadamente 2.41. Algunos empleados reportan altos niveles de satisfacción (máximo de 10), mientras que otros están menos satisfechos (mínimo de 0.9).

- **Semestre**: La variable refleja la antigüedad promedio de los empleados en la empresa, con una media de aproximadamente 200 semestres. Esto sugiere que, en promedio, los empleados han acumulado una considerable experiencia en la empresa a lo largo del tiempo.

- **Accidente**: Alrededor del 15% de los empleados ha experimentado accidentes en el trabajo, lo que subraya la importancia de las prácticas de seguridad en el lugar de trabajo y la prevención de accidentes.

- **Renunciar**: La media de 0.17 indica que un pequeño porcentaje de empleados ha tomado la decisión de renunciar a la empresa.

- **Ascenso en los últimos cinco años**: Solo alrededor del 2% de los empleados ha logrado ascender en los últimos cinco años, lo que podría sugerir áreas de mejora en oportunidades de desarrollo y promoción interna.

- **Dept_X**: Las medias proporcionan una visión general de la proporción de empleados en cada departamento, lo que puede ser útil para evaluar la distribución de la fuerza laboral en la empresa.

- **Sal_X**: Las medias ofrecen información sobre la proporción de empleados en diferentes categorías salariales, lo que puede ser relevante para la gestión de la compensación y la retención del talento.

#### 4. Estimar el modelo de Cox e interpretar cada uno de los coeficientes

In [16]:
cph1 = CoxPHFitter().fit(desercion_laboral, 'semestre', 'renunciar')
cph1.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-27 05:48:25 UTC

---
                              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                           
satisfaccion                 -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion                   -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos                    -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes            0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente                    -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years               -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
dept_Investigación           -0.35       0.70       0.12            -0.58            -0.12                 0.56                 0.89
dept_administración          -0.13       0.88       0.15            -0.42             0.16                 0.66                 1.17
dept_contabilidad            -0.11       0.90       0.10            -0.32             0.10                 0.73                 1.10
dept_desarrollo de producto   0.01       1.01       0.10            -0.20             0.21                 0.82                 1.24
dept_informática             -0.06       0.94       0.09            -0.24             0.12                 0.79                 1.13
dept_mantenimiento            0.08       1.08       0.07            -0.05             0.21                 0.95                 1.24
dept_mercadotecnia           -0.01       0.99       0.10            -0.22             0.19                 0.81                 1.21
dept_recursos humanos         0.08       1.09       0.10            -0.12             0.29                 0.89                 1.33
dept_soporte                  0.10       1.11       0.07            -0.04             0.24                 0.96                 1.27
sal_alto                     -1.15       0.32       0.15            -1.44            -0.85                 0.24                 0.43
sal_bajo                      0.39       1.48       0.05             0.30             0.48                 1.35                 1.62

                              cmp to      z      p   -log2(p)
covariate                                                    
satisfaccion                    0.00 -26.89 <0.005     526.58
evaluacion                      0.00  -0.76   0.44       1.17
proyectos                       0.00 -14.75 <0.005     161.15
promedio_horas_mes              0.00   3.88 <0.005      13.25
accidente                       0.00 -12.41 <0.005     115.04
ascenso_5years                  0.00  -3.88 <0.005      13.21
dept_Investigación              0.00  -3.01 <0.005       8.56
dept_administración             0.00  -0.89   0.37       1.42
dept_contabilidad               0.00  -1.05   0.29       1.77
dept_desarrollo de producto     0.00   0.09   0.93       0.11
dept_informática                0.00  -0.65   0.52       0.95
dept_mantenimiento              0.00   1.23   0.22       2.18
dept_mercadotecnia              0.00  -0.11   0.91       0.14
dept_recursos humanos           0.00   0.81   0.42       1.26
dept_soporte                    0.00   1.42   0.16       2.67
sal_alto                        0.00  -7.67 

In [17]:
cph1.params_

covariate
satisfaccion                  -0.242337
evaluacion                    -0.010687
proyectos                     -0.291606
promedio_horas_mes             0.001839
accidente                     -1.246686
ascenso_5years                -1.380084
dept_Investigación            -0.350547
dept_administración           -0.130262
dept_contabilidad             -0.110158
dept_desarrollo de producto    0.009468
dept_informática              -0.058469
dept_mantenimiento             0.081424
dept_mercadotecnia            -0.011698
dept_recursos humanos          0.083791
dept_soporte                   0.100538
sal_alto                      -1.147612
sal_bajo                       0.391289
Name: coef, dtype: float64

Prueba de hipótesis:

- **Ho:** La variable no tiene efecto.
- **Ha:** La variable tiene efecto.

Para las siguientes variables: **evaluación**, **dept_administración**, **dept_contabilidad**, **dept_desarrollo de producto**, **dept_informática**, **dept_mantenimiento**, **dept_mercadotecnia**, **dept_recursos humanos**, y **dept_soporte**, no se encontró evidencia estadística significativa de que tengan un efecto en el riesgo de renunciar, dado que el valor de p (p-value) no es menor que el nivel de significancia alfa.

Sin embargo, para las siguientes variables, se rechaza la hipótesis nula (Ho), lo que indica que tienen un efecto en el riesgo de renunciar:

**Satisfacción:**
- **p-value < alfa.** La satisfacción del empleado con su trabajo reduce el riesgo de renunciar en un 22%.

**Proyectos:**
- **p-value < alfa.** La cantidad de proyectos asignados al empleado reduce el riesgo de renunciar en un 25%.

**Promedio de horas al mes:**
- **p-value < alfa.** Cada hora adicional trabajada al mes aumenta el riesgo de renunciar en un 0.18%.

**Accidente:**
- **p-value < alfa.** Haber sufrido un accidente dentro de la empresa reduce el riesgo de renunciar en un 71% en comparación con no haber tenido accidentes.

**Ascenso en los últimos 5 años:**
- **p-value < alfa.** Haber sido ascendido durante los últimos 5 años reduce el riesgo de renunciar en un 75% en comparación con no haber sido ascendido.

**Departamento de Investigación:**
- **p-value < alfa.** Trabajar en el departamento de investigación tiene un 30% menos de riesgo de renunciar en comparación con trabajar en el departamento de ventas.

**Salario Alto:**
- **p-value < alfa.** Tener un salario alto conlleva un 68% menos de riesgo de renunciar en comparación con tener un salario medio.

**Salario Bajo:**
- **p-value < alfa.** Tener un salario bajo aumenta en un 48% el riesgo de renunciar en comparación con tener un salario medio.

#### 5. Con base en la interpretación de los coeficientes, ¿qué recomendación podría hacerle a la empresa para que disminuya la cantidad de renuncias de los empleados?

1. **Mejorar la Satisfacción Laboral:** Dado que la satisfacción del empleado con su trabajo ha demostrado ser un factor importante, la empresa podría enfocarse en mejorar la satisfacción de los empleados. Esto podría incluir brindar oportunidades para el desarrollo profesional, reconocimiento y recompensas, y crear un ambiente de trabajo positivo.

2. **Asignación Equilibrada de Proyectos:** La cantidad de proyectos asignados al empleado también influye en la retención. Para reducir las renuncias, la empresa podría equilibrar la carga de trabajo y asegurarse de que los empleados no estén sobrecargados, lo que podría mantenerlos comprometidos.

3. **Gestión de Horas de Trabajo:** Aunque el efecto del promedio de horas mensuales trabajadas es leve, es importante gestionar las horas de trabajo de los empleados para evitar el agotamiento y el exceso de trabajo.

4. **Mantener un Ambiente Seguro:** Dado que haber sufrido un accidente dentro de la empresa reduce el riesgo de renunciar, la empresa debería seguir enfocándose en medidas de seguridad y prevención de accidentes para aumentar la retención de empleados.

5. **Promoción y Desarrollo Profesional:** La empresa podría implementar programas de desarrollo profesional y oportunidades de promoción interna para motivar a los empleados y retener el talento. El hecho de haber sido ascendido en los últimos 5 años reduce significativamente el riesgo de renunciar.

6. **Revisar la Estructura de Compensación:** Tener un salario alto se ha asociado con un menor riesgo de renunciar, mientras que tener un salario bajo se ha asociado con un mayor riesgo. La empresa podría revisar su estructura de compensación para garantizar que los salarios sean competitivos en el mercado y ofrezcan un incentivo para que los empleados se queden. También podría considerar programas de incentivos basados en el rendimiento.


#### 6. Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado

In [18]:
cph1.check_assumptions(desercion_laboral, p_value_threshold=0.05)
## Ho: Se viola supuesto de riesgo prop. cte.
## Ha: No se viola supuesto de riesgo prop. cte.

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                                  test_statistic      p  -log2(p)
accidente                   km              0.03   0.87      0.21
                            rank            1.14   0.29      1.81
ascenso_5years              km              3.66   0.06      4.17
                            rank            6.71   0.01      6.71
dept_Investigación          km              1.04   0.31      1.70
                            rank            4.36   0.04      4.76
dept_administración         km              3.66   0.06      4.17
                            rank            1.53   0.22      2.21
dept_contabilidad           km              0.52   0.47      1.09
                            rank            0.71   0.40      1.33
dept_desarrollo de producto km              0.07   0.79      0.34
                            rank            1.10   0.29      1.77
dept_informática            km              0.08   0.77      0.37
                            rank            2.59   0.11      3.22
dept_mantenimiento          km              0.31   0.58      0.79
                            rank            0.91   0.34      1.56
dept_mercadotecnia          km              1.24   0.27      1.91
                            rank            1.31   0.25      1.99
dept_recursos humanos       km              0.00   0.99      0.01
                            rank            0.00   1.00      0.00
dept_soporte                km              2.68   0.10      3.30
                            rank            3.25   0.07      3.81
evaluacion                  km            443.05 <0.005    324.32
                            rank          427.15 <0.005    312.82
promedio_horas_mes          km            270.84 <0.005    199.74
                            rank          307.41 <0.005    226.21
proyectos                   km            574.13 <0.005    419.05
                            rank          684.93 <0.005    499.11
sal_alto                    km              0.16   0.69      0.53
                            rank            0.10   0.75      0.42
sal_bajo                    km              0.44   0.51      0.98
                            rank            0.75   0.39      1.37
satisfaccion                km           1257.74 <0.005    912.74
                            rank          845.24 <0.005    614.90



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

Este modelo de Cox no cumple el supuesto de riesgo proporcional constante, debemos utilizar AFT "modelo de riesgo acelerado"

In [19]:
## Exponencial, Weibull o LogNormal?
mexpo = ExponentialFitter().fit(desercion_laboral['semestre'], desercion_laboral['renunciar'])
mweibull = WeibullFitter().fit(desercion_laboral['semestre'], desercion_laboral['renunciar'])
mlogn = LogNormalFitter().fit(desercion_laboral['semestre'], desercion_laboral['renunciar'])

In [20]:
## Criterio AIC
mexpo.AIC_.round(2), mweibull.AIC_.round(2), mlogn.AIC_.round(2)

(15965.46, 12511.51, 11587.92)

utilizaremos para el AFT el modelo Log Normal ya que es el mejor modelo.

In [21]:
weibullAFT = LogNormalAFTFitter().fit(desercion_laboral, 'semestre', 'renunciar')
weibullAFT.print_summary()

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-27 05:48:59 UTC

---
                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                           
mu_    accidente                     0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years                0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       dept_Investigación            0.06       1.06       0.03             0.00             0.11                 1.00                 1.12
       dept_administración           0.11       1.11       0.03             0.04             0.17                 1.04                 1.18
       dept_contabilidad             0.01       1.01       0.03            -0.04             0.06                 0.97                 1.07
       dept_desarrollo de producto   0.01       1.01       0.02            -0.04             0.05                 0.96                 1.06
       dept_informática              0.01       1.01       0.02            -0.03             0.05                 0.97                 1.05
       dept_mantenimiento           -0.02       0.98       0.02            -0.05             0.01                 0.95                 1.01
       dept_mercadotecnia            0.02       1.02       0.03            -0.03             0.07                 0.97                 1.07
       dept_recursos humanos        -0.03       0.97       0.03            -0.08             0.02                 0.92                 1.02
       dept_soporte                 -0.03       0.97       0.02            -0.06             0.01                 0.94                 1.01
       evaluacion                    0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes           -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                     0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       sal_alto                      0.29       1.33       0.03             0.23             0.34                 1.26                 1.41
       sal_bajo                     -0.08       0.92       0.01            -0.11            -0.06                 0.90                 0.94
       satisfaccion                  0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept                     1.00       2.71       0.03             0.94             1.05                 2.57                 2.87
sigma_ Intercept                    -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                     cmp to      z      p   -log2(p)
param  covariate                                                    
mu_    accidente                       0.00  12.98 <0.005     125.59
       ascenso_5years                  0.00   5.80 <0.005      27.19
       dept_Investigación              0.00   2.13   0.03       4.92
       dept_administración             0.00   3.22 <0.005       9.60
       dept_contabilidad               0.00   0.56   0.58       0.79
       dept_desarrollo de producto     0.00   0.22   0.82       0.28
       dept_informática                0.00   0.51   0.61       0.71
       dept_mantenimiento              0.00  -1.25   0.21       2.25
       dept_mercadotecnia

#### 7. Interprete los coeficientes del modelo AFT

Por supuesto, aquí tienes la información reescrita:

Prueba de hipótesis: $p-value < \alpha$

- **Ho:** La variable no tiene efecto.
- **Ha:** La variable tiene efecto.

Las variables **dept_contabilidad**, **dept_desarrollo de producto**, **dept_informática**, **dept_mantenimiento**, **dept_mercadotecnia**, **dept_recursos humanos**, **dept_soporte**, y **evaluación** no tienen un efecto significativo en el riesgo de renunciar, ya que el valor de p (p-value) no es menor que el nivel de significancia alfa.

Sin embargo, para las siguientes variables, se rechaza la hipótesis nula (Ho), lo que indica que tienen un efecto en el riesgo de renunciar:

**Accidente**:
- **p-value < alfa.** Se rechaza Ho, lo que significa que haber tenido un accidente dentro de la empresa aumenta el riesgo de que el empleado renuncie en un 30% en comparación con no haber tenido accidentes.

**Ascenso en los últimos 5 años**:
- **p-value < alfa.** Se rechaza Ho, lo que indica que haber sido ascendido durante los últimos 5 años incrementa el riesgo de que el empleado renuncie en un 44% en comparación con no haber sido ascendido.

**Departamento de Investigación**:
- **p-value < alfa.** Se rechaza Ho, lo que sugiere que trabajar en el departamento de investigación incrementa el riesgo de que el empleado renuncie en un 6% en comparación con trabajar en el departamento de ventas.

**Departamento de Administración**:
- **p-value < alfa.** Se rechaza Ho, indicando que trabajar en el departamento de administración aumenta el riesgo de que el empleado renuncie en un 11% en comparación con trabajar en el departamento de ventas.

**Promedio de Horas Mensuales Trabajadas**:
- **p-value < alfa.** Se rechaza Ho, lo que implica que el promedio de horas mensuales trabajadas disminuye el riesgo de que el empleado renuncie en un 0.01%.

**Cantidad de Proyectos**:
- **p-value < alfa.** Se rechaza Ho, lo que sugiere que la cantidad de proyectos asignados al empleado aumenta el riesgo de renunciar en un 10%.

**Salario Alto**:
- **p-value < alfa.** Se rechaza Ho, lo que indica que tener un salario alto aumenta el riesgo de renunciar en un 33% en comparación con tener un salario medio.

**Salario Bajo**:
- **p-value < alfa.** Se rechaza Ho, lo que significa que tener un salario bajo disminuye el riesgo de que el empleado renuncie en un 8% en comparación con tener un salario medio.

**Satisfacción Laboral**:
- **p-value < alfa.** Se rechaza Ho, lo que sugiere que la satisfacción del empleado con su trabajo aumenta el riesgo de que renuncie en un 7%.

